In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.optimizers import *
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.callbacks import Callback
from dotmap import DotMap
import logging
import GPy
import GPyOpt
from function import *
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, MaxPool2D, Dropout, Flatten, Dense
from tensorflow.keras import Model
from tensorflow.keras.models import load_model
from sklearn.metrics import mean_squared_error, mean_absolute_error
import scipy as sp
import math
import matplotlib.pyplot as plt
%matplotlib inline

np.set_printoptions(threshold=np.inf) 
tf.compat.v1.disable_eager_execution()

2022-04-24 19:23:24.746195: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/ohpc/pub/mpi/openmpi3-gnu8/3.1.4/lib:/opt/ohpc/pub/compiler/gcc/8.3.0/lib64:/usr/local/cuda/lib64
2022-04-24 19:23:24.746225: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
# create logger with 'Model_application'
logger = logging.getLogger('Model')
logger.setLevel(logging.DEBUG)
# create file handler which logs even debug messages
fh = logging.FileHandler('CRISRPoff.log')
fh.setLevel(logging.DEBUG)
# create formatter and add it to the handlers
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
fh.setFormatter(formatter)
logger.addHandler(fh)

In [3]:
FILE = pd.read_csv("./dataset/testfile.csv")
data = pd.DataFrame(columns=(['40mer']))
data['40mer'] = FILE['40mer']

x_data = data.iloc[:, 0]
x_data = grna_preprocess(x_data,40)

y_data = FILE['efficiency']
y_data = np.array(y_data)
y_data = y_data.reshape(len(y_data), -1)

tss1,tss2,tss3,tss4 = FILE['nor_tss1'],FILE['nor_tss2'],FILE['nor_tss3'],FILE['nor_tss4']
tss1,tss2,tss3,tss4 = epi_progress(tss1),epi_progress(tss2),epi_progress(tss3),epi_progress(tss4)
Methylation,ATAC,RNA = FILE['nor_methylation'],FILE['nor_atac'],FILE['nor_rna']
Methylation,ATAC,RNA = epi_progress(Methylation),epi_progress(ATAC),epi_progress(RNA)
model_input = np.concatenate((x_data, tss1, tss2, tss3, tss4, Methylation, ATAC, RNA), axis=3)
x_train,x_test,y_train,y_test = train_test_split(model_input, y_data, test_size=0.1, random_state=1)

In [4]:
def EpiCasDL(batch_size = 60, epochs = 60,drop1 = 0.3, drop2 = 0.3, drop3 = 0.3,drop4 = 0.3, filters1 = 40, filters2 = 40, filters3 = 40,
             filters4 = 40, filters5 = 40, full_Dense1 = 80, full_Dense2 = 60, full_Dense3 = 40, optimizer = Adam, learning_rate = 0.001,
            activation = 'relu'):
    data_input = Input(shape = (1,40,11))
    data_Conv1 = Conv2D(filters = filters1, kernel_size = (1,1),padding = 'same', activation = 'relu')(data_input)
    data_Conv2 = Conv2D(filters = filters2, kernel_size = (1,2),padding = 'same', activation = 'relu')(data_input)
    data_Conv3 = Conv2D(filters = filters3, kernel_size = (1,3),padding = 'same', activation = 'relu')(data_input)
    data_Conv4 = Conv2D(filters = filters4, kernel_size = (1,4),padding = 'same', activation = 'relu')(data_input)
    data_Conv5 = Conv2D(filters = filters5, kernel_size = (1,5),padding = 'same', activation = 'relu')(data_input)
    data_t = tf.keras.layers.Concatenate()([data_Conv1, data_Conv2, data_Conv3, data_Conv4, data_Conv5])
    data_p = MaxPool2D(strides = 2, padding = 'same')(data_t)
    data_d = Dropout(drop1)(data_p)
    
    flatten = Flatten()(data_d)
    f1 = Dense(full_Dense1,activation = 'relu')(flatten)
    data_d2 = Dropout(drop2)(f1)
    f2 = Dense(full_Dense2,activation = 'relu')(data_d2)
    data_d3 = Dropout(drop3)(f2)
    f3 = Dense(full_Dense3,activation = 'relu')(data_d3)
    data_d4 = Dropout(drop4)(f3)
    output = Dense(1,activation = "linear", name = "output")(data_d4)
    model = Model(inputs = [data_input], outputs = [output])
    
    model.compile(optimizer = Adam(lr=0.0001), loss = 'mse',metrics=['mse'])
    early_stopping = EarlyStopping(monitor = 'val_loss', patience=5, verbose = 1)
    get_best_model = GetBest('./model/'+'CNN.h5', monitor = 'val_loss', verbose = 1, mode = 'min')
    model.fit(x_train,y_train,
             batch_size = batch_size,
             epochs = epochs,
             verbose = 2,
             validation_split = 0.2,
             callbacks = [get_best_model,early_stopping])
    return model

In [5]:
# string parameters changed to dictionary which keys are integers
fc_activation_dict = {'1':'relu','2':'tanh', '3':'sigmoid', '4':'hard_sigmoid', '0':'elu'}
optimizer_dict = {'1':SGD,'2':RMSprop, '3':Adagrad, '4':Adadelta,'5':Adam,'6':Adamax,'0':Nadam}
#parameter space
bounds = [
          #Discrete  
          {'name':'drop1','type':'discrete','domain':(0.1, 0.2, 0.3, 0.4, 0.5)},
          {'name':'drop2','type':'discrete','domain':(0.1, 0.2, 0.3, 0.4, 0.5)},
          {'name':'drop3','type':'discrete','domain':(0.1, 0.2, 0.3, 0.4, 0.5)},
          {'name':'drop4','type':'discrete','domain':(0.1, 0.2, 0.3, 0.4, 0.5)},
          {'name':'learning_rate','type':'discrete','domain':(0.1, 0.01, 0.02, 0.001, 0.002)},
          #Discrete
          {'name':'batch_size', 'type': 'discrete','domain': (64,128, 256, 512)},
          {'name':'epochs', 'type': 'discrete','domain': (20, 25, 30, 35, 40, 45, 50, 55,60)},
          {'name':'filters1', 'type': 'discrete','domain': (20, 30, 40, 50, 60)},
          {'name':'filters2', 'type': 'discrete','domain': (20, 30, 40, 50, 60)},
          {'name':'filters3', 'type': 'discrete','domain': (20, 30, 40, 50, 60)},
          {'name':'filters4', 'type': 'discrete','domain': (20, 30, 40, 50, 60)},
          {'name':'filters5', 'type': 'discrete','domain': (20, 30, 40, 50, 60)},
          {'name':'full_Dense1', 'type': 'discrete','domain': (40, 60, 80, 90, 100, 150, 200, 250, 300)},
          {'name':'full_Dense2', 'type': 'discrete','domain': (40, 60, 80, 90, 100, 150, 200, 250, 300)},
          {'name':'full_Dense3', 'type': 'discrete','domain': (40, 60, 80, 90, 100, 150, 200, 250, 300)},
          #Categorical
          {'name':'activation', 'type': 'categorical','domain':tuple(map(int,tuple(fc_activation_dict.keys())))},
          {'name':'optimizer', 'type': 'categorical','domain':tuple(map(int,tuple(optimizer_dict.keys())))}
         ]

In [6]:
def cat_decode(x):
    opt= DotMap()
    opt.drop1 = float(x[:, 0])
    opt.drop2 = float(x[:, 1])
    opt.drop3 = float(x[:, 2])
    opt.drop4 = float(x[:, 3])
#     opt.learning_rate = float(x[:, 4])
    
    opt.batch_size = int(x[:, 5])
    opt.epochs = int(x[:, 6]) 
    opt.filters1 =int(x[:, 7])
    opt.filters2 = int(x[:, 8])
    opt.filters3 = int(x[:, 9])
    opt.filters4 = int(x[:, 10])
    opt.filters5 = int(x[:, 11])
    opt.full_Dense1 = int(x[:, 12])
    opt.full_Dense2 = int(x[:, 13])
    opt.full_Dense3 = int(x[:, 14]) 
    
#     opt.activation = int(x[:,15])
    opt.optimizer = int(x[:,16])
    return opt

In [7]:
def f(x):
    opt = cat_decode(x)
    param = {'drop1':opt.drop1,
           'drop2':opt.drop2,
           'drop3':opt.drop3,
           'drop4':opt.drop4,
           'learning_rate':opt.learning_rate,

           'batch_size':opt.batch_size,
           'epochs':opt.epochs, 
           'filters1':opt.filters1,
           'filters2':opt.filters2,
           'filters3':opt.filters3,
           'filters4':opt.filters4,
           'filters5':opt.filters5,
           'full_Dense1':opt.full_Dense1,
           'full_Dense2':opt.full_Dense2,
           'full_Dense3':opt.full_Dense3, 

           'activation':opt.fc_activation,
           'optimizer':opt.optimizer}

    model = EpiCasDL(**param)
    y_test_pred = model.predict(x_test)
    evaluation = mean_squared_error(y_test, y_test_pred)
    logger.info('----------')
    logger.info('params:{}'.format(param))
    logger.info('metrics——mse:{},spearman:{}'.format(evaluation,sp.stats.spearmanr(y_test, y_test_pred)[0]))
    return evaluation

In [8]:
class GetBest(Callback):
    def __init__(self,filepath=None, monitor='val_loss', save_best=False,verbose=0,
                 mode='auto', period=1):
        super(GetBest, self).__init__()
        self.monitor = monitor
        self.verbose = verbose
        self.period = period
        self.save_best = save_best
        self.filepath = filepath
        self.best_epochs = 0
        self.epochs_since_last_save = 0

        if mode not in ['auto', 'min', 'max']:
            warnings.warn('GetBest mode %s is unknown, '
                          'fallback to auto mode.' % (mode),
                          RuntimeWarning)
            mode = 'auto'

        if mode == 'min':
            self.monitor_op = np.less
            self.best = np.Inf
        elif mode == 'max':
            self.monitor_op = np.greater
            self.best = -np.Inf
        else:
            if 'acc' in self.monitor or self.monitor.startswith('fmeasure'):
                self.monitor_op = np.greater
                self.best = -np.Inf
            else:
                self.monitor_op = np.less
                self.best = np.Inf
                
    def on_train_begin(self, logs=None):
        self.best_weights = self.model.get_weights()

    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        self.epochs_since_last_save += 1
        if self.epochs_since_last_save >= self.period:
            self.epochs_since_last_save = 0
            filepath = self.filepath.format(epoch=epoch + 1, **logs)
            current = logs.get(self.monitor)
            if current is None:
                warnings.warn('Can pick best model only with %s available, '
                              'skipping.' % (self.monitor), RuntimeWarning)
            else:
                if self.monitor_op(current, self.best):
                    if self.verbose > 0:
                        print('\nEpoch %05d: %s improved from %0.5f to %0.5f,'
                              ' storing weights.'
                              % (epoch + 1, self.monitor, self.best,
                                 current))
                    self.best = current
                    self.best_epochs = epoch + 1
                    self.best_weights = self.model.get_weights()
                    #self.model.save(filepath, overwrite=True)
                else:
                    if self.verbose > 0:
                        print('\nEpoch %05d: %s did not improve.' %
                              (epoch + 1, self.monitor)) 
                    
    def on_train_end(self, logs=None):
        if self.verbose > 0:
            print('Using epoch %05d with %s: %0.5f.' % (self.best_epochs, self.monitor,
                                                       self.best))
        self.model.set_weights(self.best_weights)
        #self.model.save(self.filepath, overwrite=True)

In [ ]:
#time consuming 
#set initial_design_numdata and  max_iter as lower integer to reduce searching time
opt_model = GPyOpt.methods.BayesianOptimization(f=f, domain=bounds, initial_design_numdata=30)
opt_model.run_optimization(max_iter = 300)
logger.info("optimized loss: {0}".format(opt_model.fx_opt))
for i,v in enumerate(bounds):
    name = v['name']
    logger.info('parameter {}:{}'.format(name,opt_model.x_opt[i]))